<a href="https://colab.research.google.com/github/ashwinsabu/Ashwin-Sabu/blob/master/Project_Naive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Naive Bayes Classifier

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import accuracy_score
from scipy.sparse import hstack

In [3]:
# Load the CSV file into a DataFrame
file_path = 'job_descriptions_with_keywords.csv'  # Replace with the actual path
data = pd.read_csv(file_path)

In [9]:
def convert_salary_range(salary_range):
    salary_range = salary_range.replace('$', '')
    salary_range = salary_range.replace('K', '000')
    parts = salary_range.split('-')
    min_salary = int(parts[0].replace(',', ''))
    max_salary = int(parts[1].replace(',', ''))
    return (min_salary + max_salary) / 2


In [5]:
data['Salary Range'] = data['Salary Range'].apply(convert_salary_range)

In [6]:
le_experience = LabelEncoder()
le_qualifications = LabelEncoder()
data['Experience'] = le_experience.fit_transform(data['Experience'])
data['Qualifications'] = le_qualifications.fit_transform(data['Qualifications'])

In [7]:
X = data[['Experience', 'Qualifications', 'Salary Range', 'skills']]
y = data['Job Title']

In [8]:
# Convert categorical target variable to numerical using LabelEncoder
le_job_title = LabelEncoder()
y = le_job_title.fit_transform(y)

In [12]:
# Split data into training and testing sets (70-30 split)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Vectorize the 'skills' column using CountVectorizer
vectorizer = CountVectorizer()
X_train_skills = vectorizer.fit_transform(X_train['skills'])
X_test_skills = vectorizer.transform(X_test['skills'])

# Scale the numerical columns using MinMaxScaler
scaler = MinMaxScaler()
X_train_num = scaler.fit_transform(X_train[['Experience', 'Qualifications', 'Salary Range']])
X_test_num = scaler.transform(X_test[['Experience', 'Qualifications', 'Salary Range']])

# Combine the numerical and vectorized text features
X_train_combined = hstack([X_train_num, X_train_skills])
X_test_combined = hstack([X_test_num, X_test_skills])

# Initialize the Naive Bayes classifier
nb_model = MultinomialNB()

# Train the Naive Bayes model
nb_model.fit(X_train_combined, y_train)

# Predict on the test data
y_test_pred = nb_model.predict(X_test_combined)

# Calculate testing accuracy
test_accuracy = accuracy_score(y_test, y_test_pred)
print(f"Testing Accuracy: {test_accuracy}")


Testing Accuracy: 0.954
